# Mangling
- Python doesn't support encapsulation. That is, there is no access modifiers
- To mimic encapsulation, especially **private**, use mangling technique

## Consenting Adults
- How to make class is entirely programmer's responsibility

In [6]:
class A:
    __x = 1 # mangling
assert '_A__x' in dir(A) # Change name 

# Property
- To support more realistic encapsulation, use property
- Function's access modifier is **private** and Class's access modifier is **public**
- Use function to seal properties
- To check properties
  - Use ```object.__dict__``` or ```vars(object)``` for object properties
  - Use ```Class.__dict__``` or ```vars(class)``` for class properties

In [141]:
class A:
    __x = 1
    @property
    def x(self):
        print('getter x')
        return self.__x
    @x.setter
    def x(self, new):
        print('setter x')
        self.__x = new
    @x.deleter
    def x(self):
        print('deleter x')
        del self.__x
        
    __y = 2
    def get_y(self):
        print('getter y')
        return self.__y
    def set_y(self, new):
        print('setter y')
        self.__y = new
    def del_y(self):
        print('deleter y')
        del self.__y
    y = property(get_y, set_y, del_y, "y's docstring")
    
a = A()
assert vars(a) == {} #encapsulation is acquired. y and x is private

# Method
## Class Method
- if using decorator **@classmethod**, the first argument of method refers class itself
```
class A:
    @classMethod
    def f(cls, *args):
        pass
```
- **cls** is just a convention

## Static Method
- If using decorator **@staticmethod**, the function is treated as static function
- Class is used for namespace, decorator can be omitted
  - No instnace is generated
  
## Instance Method
- If passing **instance** itself, the function is treated as instance method
```
class A:
    def f(self):
        pass
```
- **self** is just a convention

In [119]:
class A:
    x = 1
    @staticmethod
    def f():
        print('static Method')
    @classmethod
    def g(thisIsClass):
        print('class Method. x is', thisIsClass.x)
    def h(thisIsInstancd):
        print('instance Method')

A.f()
A.g()
A().h()
A.h(A()) #passing instance explicitly

static Method
class Method. x is 1
instance Method
instance Method


# Inheritance
```
class Derived(Base[, Base..])
```
## MRO(Method Resolution Order)

In [40]:
class Person:
    def say(self):
        return 'person say'
class Mother(Person):
    def say(self):
        return 'mother say'
class Father(Person):
    def say(self):
        return 'father say'
class Me(Mother, Father):
    def say(self):
        return 'me say'

print(Me.__mro__)
assert Me().say() == 'me say'

del Me.say
assert Me().say() == 'mother say'

class Me(Father, Mother):
    def say(self):
        return 'me say'

print(Me.__mro__)
del Me.say
assert Me().say() == 'father say'


class Person:
    def say(self):
        return 'person say'
class Mother(Person):
    def say(self):
        return super().say()
class Father(Person):
    def say(self):
        return 'father say'
class Me(Mother, Father):
    def say(self):
        return 'me say'
    
print(Me.__mro__)
del Me.say
assert Me().say() == 'father say' # super() call Person's say. so, Father is called

# MRO Error
class A:
    pass
class B(A):
    pass
class C(A):
    pass
class D(A, B, C): # B and C is derived by A but A is resolved before B or C
    pass

# MRO Error
class A:
    pass
class B:
    pass
class C(A, B):
    pass
class D(B, A):
    pass
class E(C, D): # C and D is derived by A and B but different order, so E can't resolve its MRO
    pass

(<class '__main__.Me'>, <class '__main__.Mother'>, <class '__main__.Father'>, <class '__main__.Person'>, <class 'object'>)
(<class '__main__.Me'>, <class '__main__.Father'>, <class '__main__.Mother'>, <class '__main__.Person'>, <class 'object'>)
(<class '__main__.Me'>, <class '__main__.Mother'>, <class '__main__.Father'>, <class '__main__.Person'>, <class 'object'>)


TypeError: Cannot create a consistent method resolution
order (MRO) for bases A, B, C

# MetaClass
- Class is an object. MetaClass makes the object

## type()
```
type(className, (baseClass), {properties and methoeds})
```
### Make class **dynamically**

In [74]:
A = type('A', (object,), {'x': 1, 'staticF': lambda: print('static Hello'), 'f': lambda self: print('hello')})
a = A()
print(type(a))
a.f()
A.staticF()

<class '__main__.A'>
hello
static Hello


### Make metaclass

In [77]:
class MakeCalc(type):
    def __new__(metacls, name, bases, namespace):
        namespace['desc'] = 'Calcuator'
        namespace['add'] = lambda a, b: a + b
        return type.__new__(metacls, name, bases, namespace)

Calc = MakeCalc('Calc', (), {})
c = Calc()
print(c.desc)

Calcuator


### Use for singleton

In [86]:
class Singleton(type):
    __instances = {}
    def __call__(cls, *args, **kwargs): #__call__ in cls!!
        if cls not in cls.__instances:
            cls.__instances[cls] = super().__call__(*args, **kwargs)
        return cls.__instances[cls]

class Hello(metaclass=Singleton):
    pass
a = Hello()
b = Hello()
assert a == b

## Instance creation flow
- \_\_new\_\_ => \_\_init\_\_ => \_\_call\_\_

### \_\_new\_\_
- Memory allocator
- passed by called class, arguments
```
class B(object):
    def __new__(cls):
        return super().__new__(cls) # equal to object.__new__(cls)
```

### \_\_init\_\_
- Initialize memory
- passed by called arguments

### \_\_call\_\_
- execute instance(with self) or class(with cls)
- passed by called arguments
- The initialization of metaclass is done in ```metaclass=MetaClass```
- Type's \_\_call\_\_  method calls cls's \_\_init\_\_

In [114]:
class CustomMetaClass(type):
    def __new__(cls, *args, **kwargs):
        print('metaclass new')
        return super().__new__(cls, *args, **kwargs) #cls is passed `cause allocating specific memory
    def __init__(cls, *args, **kwargs):
        print('metaclass init')
        super().__init__(*args, **kwargs)
    def __call__(cls, *args, **kwargs):
        print('metaclass call')
        return super().__call__(*args, **kwargs)


class MyClass(metaclass = CustomMetaClass): # At this time, MyClass, that is the instance of CustomMetaClass, is created
    def __init__(self):
        print('class init')
    def __call__(self):
        print('class call')
        
print('========================')
a = MyClass() # MyClass is the instance of CustomMetaClass, so, __call__ in CustomMetaClass is called
a()

assert isinstance(MyClass, CustomMetaClass)
assert isinstance(a, MyClass)


metaclass new
metaclass init
metaclass call
class init
class call


## Abstraction
- Using **ABCMeta**, make **protocol**
- Protocol is inherited by subclass
- Using **ABC**, make **Abstract Class**
- **ABC** is an instance of **ABCMeta**

In [130]:
from abc import *

class PersonProtocol(metaclass=ABCMeta):
    @abstractmethod
    def say(self):
        pass

class Student(PersonProtocol):
    def say(self):
        print('hello')
s = Student()

class Person(ABC):
    @abstractmethod
    def say(self):
        pass
class Student(Person):
    def say(self):
        print('hello')
s = Student()

assert isinstance(ABC, ABCMeta)

[abc.ABC, object]